# Task 4: RAG检索与生成管道演示

本Notebook演示如何使用RAG (Retrieval-Augmented Generation) 管道进行智能问答。

## 功能概述

- **检索增强生成**: 结合向量检索和大语言模型生成
- **端到端问答**: 从用户问题到结构化答案的完整流程
- **可溯源回答**: 每个答案都标注来源文档和页码
- **忠实性保证**: 严格基于检索到的上下文生成答案

## 1. 环境准备和依赖导入

In [ ]:
import os
import sys
import json
import time
from typing import Dict, Any, List

# 添加当前目录到Python路径
sys.path.append(os.path.dirname(os.path.abspath('.')))

# 导入自定义模块
from rag_pipeline import RAGPipeline
from llm_api_client import GenerationAPIClient
from embedding_module import EmbeddingAPIClient

print('✓ 依赖导入成功')

## 2. 检查必要文件

In [ ]:
# 检查必要文件是否存在
required_files = [
    'output/knowledge_base.index',
    'output/chunk_metadata.pkl',
    '.env'
]

print('检查必要文件:')
for file_path in required_files:
    if os.path.exists(file_path):
        print(f'✓ {file_path}')
    else:
        print(f'✗ {file_path} (缺失)')

missing_files = [f for f in required_files if not os.path.exists(f)]
if missing_files:
    print(f'\n⚠️  缺少文件: {missing_files}')
    print('请确保已完成Task 2和Task 3，并配置了.env文件')
else:
    print('\n✓ 所有必要文件都存在')

## 3. 初始化RAG管道

In [ ]:
# 初始化RAG管道
print('正在初始化RAG管道...')

try:
    rag = RAGPipeline(k=5)  # 检索Top-5相关文档
    print('\n✓ RAG管道初始化成功!')
except Exception as e:
    print(f'✗ RAG管道初始化失败: {e}')
    raise

## 4. 查看管道配置信息

In [ ]:
# 显示管道配置信息
info = rag.get_pipeline_info()

print('RAG管道配置信息:')
print('=' * 40)
for key, value in info.items():
    print(f'{key}: {value}')
print('=' * 40)

## 5. 单个问题演示

In [ ]:
# 演示单个问题的完整问答流程
demo_question = '公司在2023年的重大产品临床进展是什么？'

print(f'演示问题: {demo_question}')
print('=' * 60)

start_time = time.time()
result = rag.answer_question(demo_question)
end_time = time.time()

print(f'\n问题: {demo_question}')
print(f'答案: {result["answer"]}')
print(f'来源: {result["filename"]} (第{result["page"]}页)')
print(f'耗时: {end_time - start_time:.2f}秒')

## 6. 批量问题测试

In [ ]:
# 定义测试问题集
test_questions = [
    '公司的主要业务领域有哪些？',
    '公司有哪些重要的研发项目？',
    '公司的财务状况如何？',
    '公司的核心竞争优势是什么？',
    '公司在技术创新方面有什么突破？'
]

print(f'批量测试 {len(test_questions)} 个问题:')
print('=' * 60)

results = []
total_time = 0

for i, question in enumerate(test_questions, 1):
    print(f'\n问题 {i}: {question}')
    print('-' * 40)
    
    start_time = time.time()
    try:
        result = rag.answer_question(question)
        end_time = time.time()
        response_time = end_time - start_time
        total_time += response_time
        
        print(f'答案: {result["answer"]}')
        print(f'来源: {result["filename"]} (第{result["page"]}页)')
        print(f'耗时: {response_time:.2f}秒')
        
        results.append({
            'question': question,
            'answer': result['answer'],
            'filename': result['filename'],
            'page': result['page'],
            'response_time': response_time,
            'success': True
        })
        
    except Exception as e:
        end_time = time.time()
        response_time = end_time - start_time
        total_time += response_time
        
        print(f'✗ 处理失败: {e}')
        print(f'耗时: {response_time:.2f}秒')
        
        results.append({
            'question': question,
            'error': str(e),
            'response_time': response_time,
            'success': False
        })

print(f'\n批量测试完成，总耗时: {total_time:.2f}秒')

## 7. 结果分析和统计

In [ ]:
# 分析测试结果
total_tests = len(results)
successful_tests = sum(1 for r in results if r.get('success', False))
failed_tests = total_tests - successful_tests

avg_response_time = sum(r['response_time'] for r in results) / total_tests if total_tests > 0 else 0

print('测试结果统计:')
print('=' * 40)
print(f'总测试数: {total_tests}')
print(f'成功: {successful_tests} ({successful_tests/total_tests*100:.1f}%)')
print(f'失败: {failed_tests} ({failed_tests/total_tests*100:.1f}%)')
print(f'平均响应时间: {avg_response_time:.2f}秒')

# 显示成功的问答对
successful_results = [r for r in results if r.get('success', False)]
if successful_results:
    print(f'\n成功的问答示例 (共{len(successful_results)}个):')
    print('=' * 40)
    for i, result in enumerate(successful_results[:3], 1):
        print(f'\n{i}. 问题: {result["question"]}')
        print(f'   答案: {result["answer"][:100]}{"..." if len(result["answer"]) > 100 else ""}')
        print(f'   来源: {result["filename"]} (第{result["page"]}页)')

# 显示失败的测试
failed_results = [r for r in results if not r.get('success', False)]
if failed_results:
    print(f'\n失败的测试 (共{len(failed_results)}个):')
    print('=' * 40)
    for i, result in enumerate(failed_results, 1):
        print(f'{i}. {result["question"]} - {result.get("error", "Unknown error")}')

print('=' * 40)

## 8. 保存演示结果

In [ ]:
# 保存演示结果到JSON文件
demo_results = {
    'pipeline_info': rag.get_pipeline_info(),
    'test_results': results,
    'statistics': {
        'total_tests': total_tests,
        'successful_tests': successful_tests,
        'failed_tests': failed_tests,
        'success_rate': successful_tests/total_tests*100 if total_tests > 0 else 0,
        'average_response_time': avg_response_time
    }
}

output_file = 'task4_rag_demo_results.json'

try:
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(demo_results, f, ensure_ascii=False, indent=2)
    print(f'✓ 演示结果已保存到: {output_file}')
except Exception as e:
    print(f'✗ 保存演示结果失败: {e}')

## 9. 总结

### Task 4 完成情况

✅ **核心功能实现**:
- GenerationAPIClient: 文本生成API客户端
- RAGPipeline: 完整的检索增强生成管道
- 端到端问答流程: 查询向量化 → 检索 → 上下文构建 → 生成 → 格式化

✅ **关键特性**:
- 可配置的检索数量 (k参数)
- 结构化的提示词工程
- 忠实性保证 (严格基于上下文)
- 可溯源的答案 (文件名 + 页码)
- 完整的错误处理

✅ **输出格式**:
```json
{
  'answer': 'LLM生成的答案文本',
  'filename': '答案来源的文件名',
  'page': '答案来源的页码'
}
```

### 使用方法

```python
# 初始化RAG管道
rag = RAGPipeline()

# 问答
result = rag.answer_question('您的问题')
print(result)
```

### 技术架构

1. **EmbeddingAPIClient**: 查询向量化
2. **FAISS索引**: 高效向量检索
3. **GenerationAPIClient**: 文本生成
4. **RAGPipeline**: 整合所有组件的核心管道

Task 4 已成功实现完整的RAG推理逻辑! 🎉